# **Recurrent Neural Network for Text Classification**
*   Implementation of a recurrent neural network classifier supporting **multi-precision** training.
    *   Implementation currently supports training in either double, single, or half precision.
    *   This implies that both the computations and parameter storage are done in the specified precision.
*   Implementation of a recurrent neural network classifier supporting **mixed-precision** training.
    *   Implementation currently supports half precision computations with single precision parameter storage.
*   Implementations are based off of TensorFlow's very own example: [TensorFlow RNN Tutorial](https://www.tensorflow.org/text/tutorials/text_classification_rnn)

In [1]:
import time
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models

In [3]:
tfds.disable_progress_bar()
# Set a global random seed
tf.random.set_seed(12)

In [12]:
# Define number of training runs to compute the average training time over
NUM_TRAINING_RUNS = 3
# Values are specific to the imdb_reviews dataset
VOCAB_SIZE = 1000
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [7]:
def build_and_train(train_ds, encoder, precision):
    if precision == 'double':
        dtype = tf.float64
    elif precision == 'single':
        dtype = tf.float32
    else: # half
        dtype = tf.float16

    model = models.Sequential([
      encoder,
      layers.Embedding(
          input_dim=len(encoder.get_vocabulary()),
          output_dim=64,
          mask_zero=True,
          dtype=dtype),
      layers.Bidirectional(layers.LSTM(64), dtype=dtype),
      layers.Dense(64, activation='relu', dtype=dtype),
      layers.Dense(1, dtype=dtype)
    ])
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

    start_time = time.time()
    model.fit(train_ds, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    return model, training_time

In [17]:
def build_and_train_mixed(train_ds, encoder):
    tf.keras.mixed_precision.set_global_policy('mixed_float16')

    model = models.Sequential([
      encoder,
      layers.Embedding(
          input_dim=len(encoder.get_vocabulary()),
          output_dim=64,
          mask_zero=True),
      layers.Bidirectional(layers.LSTM(64)),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
    ])
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

    start_time = time.time()
    model.fit(train_ds, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    tf.keras.mixed_precision.set_global_policy('float32')
    return model, training_time

In [4]:
# Load dataset and split into train and test sets
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [5]:
# Create the text encoder
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [8]:
# Test run to make sure that everything is working properly before starting actual measurements
_ = build_and_train(train_dataset, encoder, precision='single')
_ = build_and_train_mixed(train_dataset, encoder)

Epoch 1/5
391/391 [==============================] - 46s 91ms/step - loss: 0.6504 - accuracy: 0.5514
Epoch 2/5
391/391 [==============================] - 25s 63ms/step - loss: 0.4343 - accuracy: 0.7874
Epoch 3/5
391/391 [==============================] - 23s 59ms/step - loss: 0.3962 - accuracy: 0.8217
Epoch 4/5
391/391 [==============================] - 23s 57ms/step - loss: 0.3385 - accuracy: 0.8531
Epoch 5/5
391/391 [==============================] - 23s 59ms/step - loss: 0.3234 - accuracy: 0.8600


In [13]:
# Train with double precision
time_double = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_double, training_time = build_and_train(train_dataset, encoder, 'double')
    time_double += training_time
accuracy_double = model_double.evaluate(test_dataset, verbose=2)[1]

Epoch 1/5
391/391 [==============================] - 51s 104ms/step - loss: 0.6556 - accuracy: 0.5470
Epoch 2/5
391/391 [==============================] - 28s 70ms/step - loss: 0.4262 - accuracy: 0.8051
Epoch 3/5
391/391 [==============================] - 25s 64ms/step - loss: 0.3518 - accuracy: 0.8460
Epoch 4/5
391/391 [==============================] - 26s 66ms/step - loss: 0.3273 - accuracy: 0.8586
Epoch 5/5
391/391 [==============================] - 27s 68ms/step - loss: 0.3155 - accuracy: 0.8659
Epoch 1/5
391/391 [==============================] - 49s 100ms/step - loss: 0.6408 - accuracy: 0.5616
Epoch 2/5
391/391 [==============================] - 30s 76ms/step - loss: 0.4907 - accuracy: 0.7030
Epoch 3/5
391/391 [==============================] - 26s 66ms/step - loss: 0.3597 - accuracy: 0.8379
Epoch 4/5
391/391 [==============================] - 26s 66ms/step - loss: 0.3322 - accuracy: 0.8523
Epoch 5/5
391/391 [==============================] - 26s 65ms/step - loss: 0.3168 - accur

In [14]:
# Train with single precision
time_single = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_single, training_time = build_and_train(train_dataset, encoder, 'single')
    time_single += training_time
accuracy_single = model_single.evaluate(test_dataset, verbose=2)[1]

Epoch 1/5
391/391 [==============================] - 40s 78ms/step - loss: 0.6376 - accuracy: 0.5794
Epoch 2/5
391/391 [==============================] - 25s 63ms/step - loss: 0.4047 - accuracy: 0.8163
Epoch 3/5
391/391 [==============================] - 24s 60ms/step - loss: 0.3382 - accuracy: 0.8501
Epoch 4/5
391/391 [==============================] - 24s 60ms/step - loss: 0.3170 - accuracy: 0.8624
Epoch 5/5
391/391 [==============================] - 25s 63ms/step - loss: 0.3110 - accuracy: 0.8659
Epoch 1/5
391/391 [==============================] - 42s 84ms/step - loss: 0.6409 - accuracy: 0.5629
Epoch 2/5
391/391 [==============================] - 25s 63ms/step - loss: 0.3959 - accuracy: 0.8170
Epoch 3/5
391/391 [==============================] - 24s 62ms/step - loss: 0.3376 - accuracy: 0.8514
Epoch 4/5
391/391 [==============================] - 24s 61ms/step - loss: 0.3196 - accuracy: 0.8615
Epoch 5/5
391/391 [==============================] - 24s 60ms/step - loss: 0.3145 - accurac

In [15]:
# Train with half precision
time_half = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_half, training_time = build_and_train(train_dataset, encoder, 'half')
    time_half += training_time
accuracy_half = model_half.evaluate(test_dataset, verbose=2)[1]

Epoch 1/5
391/391 [==============================] - 49s 100ms/step - loss: 0.7071 - accuracy: 0.5370
Epoch 2/5
391/391 [==============================] - 25s 65ms/step - loss: 0.6502 - accuracy: 0.6052
Epoch 3/5
391/391 [==============================] - 26s 65ms/step - loss: 0.5924 - accuracy: 0.6631
Epoch 4/5
391/391 [==============================] - 26s 67ms/step - loss: 0.5287 - accuracy: 0.7198
Epoch 5/5
391/391 [==============================] - 26s 65ms/step - loss: 0.4913 - accuracy: 0.7516
Epoch 1/5
391/391 [==============================] - 40s 83ms/step - loss: 0.7202 - accuracy: 0.5212
Epoch 2/5
391/391 [==============================] - 27s 67ms/step - loss: 0.6654 - accuracy: 0.5876
Epoch 3/5
391/391 [==============================] - 27s 68ms/step - loss: 0.6078 - accuracy: 0.6468
Epoch 4/5
391/391 [==============================] - 27s 68ms/step - loss: 0.5521 - accuracy: 0.7051
Epoch 5/5
391/391 [==============================] - 26s 66ms/step - loss: 0.4935 - accura

In [18]:
# Train with mixed half precision
time_mixed = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_mixed, training_time = build_and_train_mixed(train_dataset, encoder)
    time_mixed += training_time
accuracy_mixed = model_mixed.evaluate(test_dataset, verbose=2)[1]

Epoch 1/5
391/391 [==============================] - 46s 89ms/step - loss: 0.6343 - accuracy: 0.5732
Epoch 2/5
391/391 [==============================] - 27s 68ms/step - loss: 0.4400 - accuracy: 0.7841
Epoch 3/5
391/391 [==============================] - 25s 63ms/step - loss: 0.3865 - accuracy: 0.8265
Epoch 4/5
391/391 [==============================] - 25s 63ms/step - loss: 0.3609 - accuracy: 0.8466
Epoch 5/5
391/391 [==============================] - 25s 63ms/step - loss: 0.3318 - accuracy: 0.8620
Epoch 1/5
391/391 [==============================] - 41s 81ms/step - loss: 0.6504 - accuracy: 0.5641
Epoch 2/5
391/391 [==============================] - 26s 66ms/step - loss: 0.4036 - accuracy: 0.8156
Epoch 3/5
391/391 [==============================] - 25s 64ms/step - loss: 0.3357 - accuracy: 0.8525
Epoch 4/5
391/391 [==============================] - 25s 63ms/step - loss: 0.3185 - accuracy: 0.8596
Epoch 5/5
391/391 [==============================] - 25s 63ms/step - loss: 0.3092 - accurac

In [19]:
print("---RESULTS---")
print("Average training time in double precision:", time_double / NUM_TRAINING_RUNS, "seconds")
print("Average training time in single precision:", time_single/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in half precision:", time_half/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in mixed half precision:", time_mixed/ NUM_TRAINING_RUNS, "seconds")
print("-------------")
print("Accuracy with double precision:", accuracy_double)
print("Accuracy with single precision:", accuracy_single)
print("Accuracy with half precision:", accuracy_half)
print("Accuracy with mixed half precision:", accuracy_mixed)

---RESULTS---
Average training time in double precision: 203.3181173801422 seconds
Average training time in single precision: 159.53325843811035 seconds
Average training time in half precision: 180.95898509025574 seconds
Average training time in mixed half precision: 161.46620202064514 seconds
-------------
Accuracy with double precision: 0.858680009841919
Accuracy with single precision: 0.8617200255393982
Accuracy with half precision: 0.7752799987792969
Accuracy with mixed half precision: 0.8634399771690369
